In [20]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.neighbors import KNeighborsClassifier
import numpy as np

In [25]:
scrimmage_files = os.listdir('data/scrimmage_yakker')
game_files = os.listdir('data/game_yakker')
bullpen_files = os.listdir('data/pitching_yakker')

data = pd.DataFrame()

for file in scrimmage_files:
    if file.endswith('.csv'):
        df = pd.read_csv(f'data/scrimmage_yakker/{file}')
        df['type'] = 'scrimmage'
        data = pd.concat([data, df])

for file in game_files:
    if file.endswith('.csv'):
        df = pd.read_csv(f'data/game_yakker/{file}')
        df['type'] = 'game'
        data = pd.concat([data, df])

for file in bullpen_files:
    if file.endswith('.csv'):
        df = pd.read_csv(f'data/pitching_yakker/{file}')
        df['type'] = 'bullpen'
        data = pd.concat([data, df])

data = data[data['PitcherTeam'] == 'Georgia tech']

In [31]:
pitchers = pd.unique(data['Pitcher'])

for pitcher in pitchers:

    pitcher_data = data[data['Pitcher'] == pitcher].dropna(subset=['SpinAxis', 'PlateLocSide']).reset_index(drop=True)

    radians = np.deg2rad(pitcher_data['SpinAxis'])

    # Transform using sine and cosine
    pitcher_data['sin_axis'] = np.sin(radians)
    pitcher_data['cos_axis'] = np.cos(radians)

    train = pitcher_data[~pitcher_data['TaggedPitchType'].isna()]
    test = pitcher_data[pitcher_data['TaggedPitchType'].isna()]

    knn = KNeighborsClassifier(n_neighbors=len(train['TaggedPitchType'].unique()))
    knn.fit(train[['RelSpeed', 'HorzBreak', 'InducedVertBreak', 'sin_axis', 'cos_axis']], train['TaggedPitchType'])

    test['TaggedPitchType'] = knn.predict(test[['RelSpeed', 'HorzBreak', 'InducedVertBreak', 'sin_axis', 'cos_axis']])

    pitcher_data = pd.concat([train, test])
    pitcher_data = pitcher_data[pitcher_data['type'] != 'bullpen']

    for pitch in pitcher_data['TaggedPitchType'].unique():
        pitch_data = pitcher_data[(pitcher_data['TaggedPitchType'] == pitch) & (pitcher_data['PlateLocSide'] > -2) & (pitcher_data['PlateLocSide'] < 2) & (pitcher_data['PlateLocHeight'] > 0) & (pitcher_data['PlateLocHeight'] < 5)]

        # do a density plot of the called strikes
        plt.scatter(pitch_data['PlateLocSide'], pitch_data['PlateLocHeight'], s=1)
        sns.kdeplot(data=pitch_data, x='PlateLocSide', y='PlateLocHeight', levels = 6, cmap='coolwarm', fill=True, alpha=.5, linewidths=0)
        plt.plot([-17/24, 17/24], [5/4, 5/4], color='black')
        plt.plot([-17/24, 17/24], [3, 3], color='black')
        plt.plot([-17/24, -17/24], [5/4, 3], color='black')
        plt.plot([17/24, 17/24], [5/4, 3], color='black')
        plt.plot([29/24, 29/24], [-1, 6], color='black', linestyle='dotted')
        plt.plot([-29/24, -29/24], [-1, 6], color='black', linestyle='dotted')
        plt.xlim(-2, 2)
        plt.ylim(0, 5)
        plt.xticks([])
        plt.yticks([])
        plt.xlabel('')
        plt.ylabel('')
        plt.title(f'{pitcher} - {pitch}', fontsize=8)
        plt.gcf().set_size_inches(1.9, 2.8)
        plt.savefig('heatmaps/' + pitcher + ' - ' + pitch + '.png', dpi=300)
        plt.close()

/var/folders/gp/wmz27q392sx2zj_8qbdfn0jh0000gn/T/ipykernel_78850/3457425190.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['TaggedPitchType'] = knn.predict(test[['RelSpeed', 'HorzBreak', 'InducedVertBreak', 'sin_axis', 'cos_axis']])
/Users/tysonking/anaconda3/lib/python3.11/site-packages/seaborn/distributions.py:1185: UserWarning: linewidths is ignored by contourf
  cset = contour_func(
/Users/tysonking/anaconda3/lib/python3.11/site-packages/seaborn/distributions.py:1185: UserWarning: linewidths is ignored by contourf
  cset = contour_func(
/Users/tysonking/anaconda3/lib/python3.11/site-packages/seaborn/distributions.py:1185: UserWarning: linewidths is ignored by contourf
  cset = contour_func(
/Users/tysonking/anaconda3/lib/python3.11